In [98]:

import math  
import numpy as np   
import random  as rd
import matplotlib.pyplot as plt  
from matplotlib import colors
import numba  
from numba import njit  
import time  
from matplotlib.pyplot import figure

from numba.typed import List   
from collections import defaultdict   
import random as rd

from scipy.sparse import csr_matrix
from scipy import linalg


In [99]:
n1=1000
n2=1600
r=10
ratio = 0.3
def  create_fake_data():
    K= np.random.rand(n1, r)
    L= np.random.rand(r, n2)
    return K@L
M = create_fake_data()
# Omega  =  [ ]
p = round(n1*n2*ratio)
# for i in range(p):
#     Omega.append( [ rd.randint (0, n1-1),rd.randint (0, n2-1) ])
# Omega =  np.array (Omega, np.int_)

def get_m_random_index_of_matrix(n_1, n_2, m):
    return rd.sample([(i, j) for i in range(n_1) for j in range(n_2)], m)
Omega = np.array(get_m_random_index_of_matrix(n1, n2, p))

b = np.empty( (p), np.float_)
for i in range(p):
    b[i] = M[Omega[i,0], Omega[i,1]]

In [100]:
max(M.flatten())

6.614007898526751

In [101]:
index = [  o[0]*n2+ o[1] for o in Omega]
index
A= csr_matrix(([1]*p, (range(p),index)), shape=(p, n1*n2)  )
A_T=  A.T

In [102]:
A.shape

(480000, 1600000)

In [103]:
A_T.shape

(1600000, 480000)

In [104]:
AA_T =A@A_T
A_TA =A_T@A
A_Tb  =  A_T@b
mu0= np.linalg.norm(A_Tb)

In [105]:
A_Tb.shape

(1600000,)

In [106]:
# X = A_Tb.reshape(n1,n2)
# a = np.linalg.matrix_rank(X)
# a

In [107]:
def vec_to_mat_X(X):
    return np.reshape(X, (n1, n2))

In [108]:
z=np.random.rand ( (n1*n2))
def nablaf_2(z):
    return A_TA@z- A_Tb
def compute_y_n( z, beta):
    return z-(1/beta)*nablaf_2(z)
nablaf_2(z)

array([ 0.        , -1.49262889,  0.        , ...,  0.        ,
       -1.62993063,  0.        ])

In [109]:
z  

array([0.58508071, 0.42812402, 0.38039841, ..., 0.01999575, 0.18359236,
       0.79226427])

In [110]:
def proximal(Z, l_k,mu):
    tau = mu/l_k
    Z = np.reshape(Z, (n1, n2))
    U,Sigma,V = linalg.svd(Z)
   

    full_sigma  = np.zeros( (n1,n2), np.float_)

    for i in range(len(Sigma)):
        if Sigma[i]>tau:
            full_sigma[i][i] = Sigma[i]-tau

   
    return ((U@full_sigma)@V).flatten()

In [111]:
z.shape

(1600000,)

In [112]:
def main_loop(z, beta=1, mu=0.0001*mu0, loop_size=1000):
    t=np.zeros( (loop_size), np.float_)
    t[0]=1
    
    Xold = proximal(z, beta,mu)
    X = Xold
 
    mu1 = mu0
    for n in range(loop_size-1):

        t[n+1]= (math.sqrt(4*t[n]*t[n]+1)+1)/2
        
        lambda_loop = (t[n]-1)/t[n+1]

        mu1= max(mu1*0.7, mu)
        
        Y = X+ lambda_loop*(X-Xold)

        Xold = X

        Z = Y - (1/beta)*nablaf_2(Y)
        
        X = proximal(Z, beta,mu1)
        
        print(n,"error",np.linalg.norm(vec_to_mat_X(X)-M)/ np.linalg.norm(M))

        stop_condition = abs(np.linalg.norm(A@ X - b ) - np.linalg.norm(A @ Xold -b)) / (max(1, np.linalg.norm(b)))
#         print(stop_condition)
        if stop_condition < 5e-5:
            count+=1
            print("sovonglap:",n)
        else: 
            count=0
        if count==5:
            break

    return X

In [113]:
z=np.random.rand ( (n1*n2))
time1=time.time()
X_new = main_loop(z, beta=1, mu=0.0001*mu0, loop_size=1000)
print(time.time() - time1 )

0 error 0.9526149291449584
1 error 0.9636094465654167
2 error 0.8675711238293138
3 error 0.7060482890599817
4 error 0.5230977508692338
5 error 0.3544255109829096
6 error 0.22329252311674042
7 error 0.14110934034598727
8 error 0.10588451023307957
9 error 0.09778876052325658
10 error 0.09701503871025789
11 error 0.09050967685529152
12 error 0.0761754105817309
13 error 0.05758913792832621
14 error 0.03910402883535193
15 error 0.023799086727803346
16 error 0.012990698175196751
17 error 0.006594521641511687
18 error 0.003703738694392952
19 error 0.002828786968134822
20 error 0.0026203075755442513
21 error 0.0025920742046602646
22 error 0.002577074450071296
sovonglap: 22
23 error 0.0024666740561825337
24 error 0.0022283081210424547
25 error 0.0018991600355688027
26 error 0.001567463335674751
27 error 0.0012530276679524764
28 error 0.0009758780237528908
29 error 0.0007564866970513515
30 error 0.0006143574417801954
31 error 0.000555550462117713
sovonglap: 31
32 error 0.0005584159228986795
sovo